In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys

import torch
import torchio as tio

dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: 
    sys.path.append(dir1)

from utils import load_module
from data_processing import *
from transforms import *
from evaluators import *
from loggers.wandb_logger import to_wandb


if torch.cuda.is_available():
    device = torch.device('cuda')
    print("CUDA is available. Using GPU.")
else:
    device = torch.device('cpu')
    print("CUDA is not available. Using CPU.")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
CUDA is available. Using GPU.


In [3]:
prediction_name = "cv1-ens2-orientations-majority"

subject_loader = ComposeLoaders([
    ImageLoader(glob_pattern="ground_truth.*", image_name="ground_truth", 
                image_constructor=tio.LabelMap, label_values={"lesion": 1}),
])
transform = CustomOneHot()
dataset = SubjectFolder("X:/Datasets/MSSEG2_processed/", subject_path="", subject_loader=subject_loader, transforms=transform)

subject_loader = ComposeLoaders([
    ImageLoader(glob_pattern=f"{prediction_name}.*", image_name=prediction_name, 
                image_constructor=tio.LabelMap, label_values={"lesion": 1}),
])
dataset.load_additional_data('X:/Predictions/MSSEG2/', subject_loader)

In [4]:
seg_eval = InstanceSegmentationEvaluator(prediction_name, 'ground_truth')
evaluation = seg_eval(dataset)
evaluation['summary_stats'] = evaluation['summary_stats'].to_dataframe()

C:\Users\Cefir\anaconda3\envs\machine_learning\lib\site-packages\torch\_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ..\aten\src\ATen\native\BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [5]:
evaluation['subject_stats']

,subject,target_components,predicted_components,target_detections,predicted_detections,detection_recall,detection_precision,detection_f1,target_volume,prediction_volume,TP,FP,TN,FN,dice,jaccard,precision,recall
0,013,1,2,1.0,1.0,1.000000,0.500000,0.666667,40.0,122.0,33.0,89.0,23920512.0,7.0,0.407407,0.255814,0.270492,0.825000
1,015,0,0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,23920640.0,0.0,NaN,NaN,NaN,NaN
2,016,7,8,6.0,4.0,0.857143,0.500000,0.631579,743.0,754.0,540.0,214.0,23919684.0,203.0,0.721443,0.564263,0.716180,0.726783
3,018,2,4,1.0,1.0,0.500000,0.250000,0.333333,364.0,299.0,170.0,129.0,18431508.0,194.0,0.512821,0.344828,0.568562,0.467033
4,019,0,0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,83886080.0,0.0,NaN,NaN,NaN,NaN
5,020,1,1,1.0,1.0,1.000000,1.000000,1.000000,201.0,256.0,152.0,104.0,40845008.0,49.0,0.665208,0.498361,0.593750,0.756219
6,021,4,7,4.0,4.0,1.000000,0.571429,0.727273,2227.0,3722.0,2045.0,1677.0,23916736.0,182.0,0.687510,0.523822,0.549436,0.918276
7,024,6,6,5.0,5.0,0.833333,0.833333,0.833333,703.0,726.0,559.0,167.0,23919770.0,144.0,0.782365,0.642529,0.769972,0.795164
8,026,9,19,7.0,7.0,0.777778,0.368421,0.500000,376.0,1066.0,268.0,798.0,5307242.0,108.0,0.371706,0.228279,0.251407,0.712766
9,027,4,6,4.0,4.0,1.000000,0.666667,0.800000,1059.0,1168.0,869.0,299.0,23919282.0,190.0,0.780422,0.639912,0.744007,0.820585


In [6]:
evaluation['summary_stats']

,summary_stat,target_components,predicted_components,target_detections,predicted_detections,detection_recall,detection_precision,detection_f1,target_volume,prediction_volume,TP,FP,TN,FN,dice,jaccard,precision,recall
0,mean,3.800000,7.40000,3.150000,2.975000,0.841780,0.586606,0.647807,1485.800049,2138.175049,1146.724976,991.450012,33058804.0,339.075012,0.548409,0.415044,0.593673,0.640782
1,std,6.013659,15.35695,5.081641,4.896297,0.233028,0.333147,0.272611,3778.051270,5198.223633,3117.016846,3309.806641,19048260.0,730.468811,0.255267,0.221482,0.263004,0.240723
2,min,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5307242.0,0.000000,0.000000,0.000000,0.000000,0.000000
3,max,32.000000,90.00000,28.000000,27.000000,1.000000,1.000000,1.000000,19704.000000,22072.000000,16257.000000,20350.000000,83886080.0,3447.000000,0.834395,0.715847,1.000000,0.932055
4,median,1.000000,2.00000,1.000000,1.000000,1.000000,0.583333,0.666667,201.000000,256.000000,152.000000,89.000000,23920532.0,104.000000,0.627081,0.456751,0.655172,0.712766
5,mode,0.000000,0.00000,0.000000,0.000000,1.000000,1.000000,0.666667,0.000000,0.000000,0.000000,0.000000,23920640.0,0.000000,0.000000,0.000000,0.000000,0.000000


In [7]:
import wandb
wandb.init(project="msseg2-evaluation", dir="X:/wandb/", name=prediction_name)
wandb.log(to_wandb(evaluation))
wandb.finish()

wandb: Currently logged in as: efirdc (use `wandb login --relogin` to force relogin)


_runtime,2
_timestamp,1627062047
_step,0


_runtime,▁
_timestamp,▁
_step,▁
